# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from openpyxl import Workbook, load_workbook

## Read Dataset

In [3]:
# function needed for reading the data
data = load_workbook("Dataset.xlsx")

# Ddefine a function for creating a dictionary of each parameter values which is appropriate for Gurobi model
def cell_extractor(cell):
    if len(cell[0]) == 1:
        return {i: cell[i][j].value for i in list(range(len(cell))) for j in list(range(len(cell[0])))}
    else:
        return {(i,j): cell[i][j].value for i in list(range(len(cell))) for j in list(range(len(cell[0])))}

In [4]:
########################################## Sets ##########################################

# 4762 residence area - h
# 6 products - i
# 14 materials - j
# 318 cities - c, drop-off
# 5 manual primary processor P_M
# 1 automated primary processor P_A
# 2 semi-automated primary processor P_S
# 28 secondary processor - s

# Index of returned products e-waste
# 0=Desktop, 1=Laptop, 2=MonCRT, 3=TVCRT, 4=MonLCD, 5=TVLCD
I = list(range(6))

# Index of residence areas
H = list(range(4762))

# Index of drop-off sites
C = list(range(318))

# Index of manual primary processors
P_M = [0,1,2,3,4]

# Index of automated primary processors
P_A = [5]

# Index of semi-automated processor
P_S = [6,7]

#  Index of primary processors
P = P_A + P_M + P_S

# Index of secondary processor
S = list(range(28))

# Index of material
J = list(range(14))

########################################## Parameters ##########################################

# q[j,i]
q = cell_extractor(data["q_ji"]['A1':'F14'])

# r[i,h]
r = cell_extractor(data["r_ih"]['A1':'GAD6'])

# pp[h]
pp = cell_extractor(data["pp_h"]['A1':'A4762'])

# df[c]
df = cell_extractor(data["df_c"]['A1':'A318'])

# hs
hs = 4

# rq[i,c]

# ty[h]
ty =  cell_extractor(data["ty_h"]['A1':'A4762'])

# nof_pri
nof_pri = 0

# nof_sec
nof_sec = 0

# nof_drp
nof_drp = 0

# pt
pt = 0.8

# re_sec[j]
re_sec = {(j): 0.3 for j in J}

# re_pri[i]
re_pri = {(i): 0.4 for i in I}

# re_drp[i]
re_drp = {(i): 0.2 for i in I}

# tcocc
0.348

# tco[h,c]
tco_res2drp = cell_extractor(data["tco_hc"]['A1':'LF4762'])

# tco[c,p]
tco_drp2pri = {(c,p): 0.348 for c in C for p in P}

# tco[p,s]
tco_pri2sec = {(p,s): 0.482 for p in P for s in S}

# d_res2drp[h,c]
d_res2drp = cell_extractor(data["d_hc"]['A1':'LF4762'])

# d_drp2pri[c,p]
d_drp2pri = cell_extractor(data["d_cp"]['A1':'H318'])

# d_pri2sec[p,s]
d_pri2sec = cell_extractor(data["d_ps"]['A1':'AB8'])

# cap_drp[i,c]
cap_drp = cell_extractor(data["cap_ic"]['A1':'LF6'])

# cap_pri[i,p]
cap_pri = cell_extractor(data["cap_ip"]['A1':'H6'])

# cap_sec[j,s]
cap_sec = cell_extractor(data["cap_js"]['A1':'AB14'])

# pco_drp[i,c]
pco_drp = cell_extractor(data["pco_ic"]['A1':'LF6'])

# pco_sec[j,s]
pco_sec = cell_extractor(data["pco_js"]['A1':'AB14'])

# pco_pri[i,p]
pco_pri = cell_extractor(data["pco_ip"]['A1':'H6'])

# env_drp[i,c]
env_drp = cell_extractor(data["env_ic"]['A1':'LF6'])

# env_pri[i,p]
env_pri = cell_extractor(data["env_ip"]['A1':'H6'])

# env_sec[j,s]
env_sec = cell_extractor(data["env_js"]['A1':'AB14'])

# fcc[c]
fcc = {(c): 100000 for c in C}

# fcp[p]
fcp = {(p): 500000 for p in P}

# fcs[s]
fcs = {(s): 200000 for s in S}

# pcr_drp[i,c]
pcr_drp = {(i,c): 1 for i in I for c in C}

# pcr_sec[j,s]
pcr_sec = {(j,s): 1 for j in J for s in S}

# pcr_pri[i,p]
pcr_pri = {(i,p): 1 for i in I for p in P}

# env_res2drp[i,h,c]
env_res2drp = {(i,h,c): 1*env_drp[i,c] for i in I for h in H for c in C}

# env_drp2pri[i,c,p]
env_drp2pri = {(i,c,p): 1*env_pri[i,p] for i in I for c in C for p in P}

# env_pri2sec[j,p,s]
env_pri2sec = {(j,p,s): 1*env_sec[j,s] for j in J for p in P for s in S}

# ecr_drp[i,c]
ecr_drp = {(i,c): 1 for i in I for c in C}

# ecr_sec[j,s]
ecr_sec = {(j,s): 1 for j in J for s in S}

# ecr_pri[i,p]
ecr_pri = {(i,p): 1 for i in I for p in P}

# lim_drp[i,c]
lim_drp = {(i,c): 0 for i in I for c in C}

# lim_sec[j,s]
lim_sec = {(j,s): 0 for j in J for s in S}

# lim_pri[i,p]
lim_pri = {(i,p): 0 for i in I for p in P}

# eff_pri[j,p]
eff_pri = {(j,p): 1 for j in J for p in P}

# System Optimum Model
Policy makers aim to design public systems that operate cost-effectively to perform a necessary function. This translates to a perspective in which optimizing the entire system with respect to some criteria defined over all components of the system is a logical goal, if a reasonable representation of the system under investigation could be modeled. In other words, the policy makers prefer to have a system with all components operating optimally. To meet such a requirement, we develop a mixed integer linear programming (MILP) model that optimizes the total cost incurred and total emission generated through the e-waste reverse logistics network.

In [5]:
from gurobipy import Model, GRB, quicksum
SysMdl = Model('System Optimum E-Waste RL')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-12


In [6]:
########################################## Decision Var. ##########################################

# The fraction of trips for shipping i from collection site (c,t,u). 
RTD = SysMdl.addVars(I, H, C, lb=0.0, vtype=GRB.CONTINUOUS) 

# Amount of product i shipped from collection site (c,t,u) to primary processor p.
DTP = SysMdl.addVars(I, C, P, lb=0.0, vtype=GRB.CONTINUOUS) 

# Amount of material j shipped from primary processor p to secondary processor to s.
PTS = SysMdl.addVars(J, P, S, lb=0.0, vtype=GRB.CONTINUOUS) 

# Equal to one if collection site (c,t,u) is opened; otherwise, zero.
X = SysMdl.addVars(C, vtype=GRB.BINARY) 

# Equal to one if primary processor p is opened; otherwise, zero.
Y = SysMdl.addVars(P, vtype=GRB.BINARY) 

# Equal to one if scondary processor s is opened; otherwise, zero.
R = SysMdl.addVars(S, vtype=GRB.BINARY) 


########################################## Objective Function ##########################################

#### Total Cost
# transportation cost
TrCo = (quicksum((pp[h]/hs)*pt*ty[h]*df[c]*tco_res2drp[h,c]*d_res2drp[h,c]*RTD[i,h,c] 
                 for i in I for h in H for c in C) +
        quicksum(tco_drp2pri[c,p]*d_drp2pri[c,p]*DTP[i,c,p] for i in I for c in C for p in P) + 
        quicksum(tco_pri2sec[p,s]*d_pri2sec[p,s]*PTS[j,p,s] for j in J for p in P for s in S))

# process cost
PrCo = (quicksum((1-re_drp[i])*r[i,h]*pco_drp[i,c]*RTD[i,h,c] for i in I for h in H for c in C) +
        quicksum((1-re_pri[i])*pco_pri[i,p]*DTP[i,c,p] for i in I for c in C for p in P) +
        quicksum((1-re_sec[j])*pco_sec[j,s]*PTS[j,p,s] for j in J for p in P for s in S))
        
# fixed cost   
FiCo = (quicksum(X[c]*fcc[c] for c in C) +
        quicksum(Y[p]*fcp[p] for p in P) +
        quicksum(R[s]*fcs[s] for s in S))

# revenue
SeRe = (quicksum(re_drp[i]*r[i,h]*RTD[i,h,c]*pcr_drp[i,c] for i in I for h in H for c in C) +
        quicksum(re_pri[i]*DTP[i,c,p]*pcr_pri[i,p] for i in I for c in C for p in P) +
        quicksum(re_sec[j]*PTS[j,p,s]*pcr_sec[j,s] for j in J for p in P for s in S))

#### Total Emission
# transportation emission
TrEm = (quicksum((pp[h]/hs)*pt*ty[h]*df[c]*env_res2drp[i,h,c]*d_res2drp[h,c]*RTD[i,h,c] 
                 for i in I for h in H for c in C) +
        quicksum(env_drp2pri[i,c,p]*d_drp2pri[c,p]*DTP[i,c,p] for i in I for c in C for p in P) + 
        quicksum(env_pri2sec[j,p,s]*d_pri2sec[p,s]*PTS[j,p,s] for j in J for p in P for s in S))

# process emission
PrEm = (quicksum((1-re_drp[i])*r[i,h]*env_drp[i,c]*RTD[i,h,c] for i in I for h in H for c in C) +
        quicksum((1-re_pri[i])*env_pri[i,p]*DTP[i,c,p] for i in I for c in C for p in P) +
        quicksum((1-re_sec[j])*env_sec[j,s]*PTS[j,p,s] for j in J for p in P for s in S))

# revenue emission
SeEm = (quicksum(re_drp[i]*r[i,h]*RTD[i,h,c]*ecr_drp[i,c] for i in I for h in H for c in C) +
        quicksum(re_pri[i]*DTP[i,c,p]*ecr_pri[i,p] for i in I for c in C for p in P) +
        quicksum(re_sec[j]*PTS[j,p,s]*ecr_sec[j,s] for j in J for p in P for s in S))


#### Set Objective
SysMdl.setObjective(TrCo + PrCo + FiCo - SeRe + TrEm + PrEm - SeEm, GRB.MINIMIZE)


########################################## Subject to. ##########################################
# flow balance (eq:FeBaC1)
#SysMdl.addConstrs(quicksum(RTD[i,h,c] for c in C) == 1 for i in I for h in H)

# flow balance (eq:FeBaC2)
SysMdl.addConstrs(quicksum(DTP[i,c,p] for p in P)  == 
                  quicksum(r[i,h]*RTD[i, h, c]*(1-re_drp[i]) for h in H) for i in I for c in C)

# flow balance (eq:FeBaC3)
SysMdl.addConstrs(quicksum(PTS[j,p,s] for s in S)  == 
                  quicksum(q[j,i]*eff_pri[j,p]*DTP[i,c,p]*(1-re_pri[i]) for i in I for c in C) for j in J for p in P)

# capacity (eq:CapC1)
SysMdl.addConstrs(quicksum(r[i,h]*RTD[i,h,c] for h in H) <= X[c]*cap_drp[i,c] for i in I for c in C) 

# capacity (eq:CapC2)
SysMdl.addConstrs(quicksum(DTP[i,c,p] for c in C) <= Y[p]*cap_pri[i,p] for i in I for p in P)

# capacity (eq:CapC3)
SysMdl.addConstrs(quicksum(PTS[j,p,s] for p in P) <= R[s]*cap_sec[j,s] for j in J for s in S)

# shipment (eq:ShipC1)
SysMdl.addConstrs(quicksum(r[i,h]*RTD[i,h,c] for h in H) >= X[c]*lim_drp[i,c] for i in I for c in C) 

# shipment (eq:ShipC2)
SysMdl.addConstrs(quicksum(DTP[i,c,p] for c in C) >= Y[p]*lim_pri[i,p] for i in I for p in P)

# shipment (eq:ShipC3)
SysMdl.addConstrs(quicksum(PTS[j,p,s] for p in P) >= R[s]*lim_sec[j,s] for j in J for s in S)

# open facilities (eq:NoFaC1)
SysMdl.addConstr(quicksum(X[c] for c in C) >= nof_drp)

# open facilities (eq:NoFaC2)
SysMdl.addConstr(quicksum(Y[p] for p in P) >= nof_pri)

# open facilities (eq:NoFaC3)
SysMdl.addConstr(quicksum(R[s] for s in S) >= nof_sec)

########################################## Run Gurobi ##########################################
SysMdl.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 48 physical cores, 96 logical processors, using up to 32 threads
Optimize a model with 6719 rows, 9104650 columns and 27528930 nonzeros
Model fingerprint: 0x489f5c38
Variable types: 9104296 continuous, 354 integer (354 binary)
Coefficient statistics:
  Matrix range     [3e-05, 1e+09]
  Objective range  [2e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 0.0000000
Presolve removed 2351 rows and 2848 columns (presolve time = 5s) ...
Presolve removed 2707 rows and 2848 columns (presolve time = 10s) ...
Presolve removed 2707 rows and 117112 columns (presolve time = 23s) ...
Presolve removed 2707 rows and 7087216 columns (presolve time = 25s) ...
Presolve removed 4631 rows and 9088760 columns (presolve time = 30s) ...
Presolve removed 4623 rows and 90